# Классическое компьютерное зрение, практическая работа

Этот ноутбук содержит примеры работы с библиотекой OpenCV. Он посвящен цветовым пространствам, а также построению и анализу бинарных масок объектов.

**Примечание для преподавателей.**

Задания приведены с решениями, их стоит предварительно удалять, а потом заниматься написанием кода вместе с учениками. Если что-то не получается, лучше сначала всем посоветоваться, потом погуглить, а если ничего из этого не помогло, нужно обращаться к этому ноутбуку.

## Часть 1: распределения

Обратите внимание на изменения, внесенные в код.

Теперь при изменении значения трекбара новое значение не будет распечатываться.


**Задание 1:** добавьте в программу возможность останавливать считывание новых кадров по нажатию пробела (функцию паузы воспроизведения). Чтобы сделать это, нужно во-первых узнать код клавиши "пробел" (можно распечатать *key*), во-вторых завести булевую переменную (флаг) того, нужно ли считывать новые кадры, и в-третьих завести переменную для кадра, в которой он будет храниться, пока считывание новых не производится.

**Задание 2:** замените прибавление значения *val* ко всем каналам всех пикселей на прибавление только к нулевому каналу. Как меняются цвета при изменении положения трекбара? Попробуйте прибавлять его значение ко второму каналу, к третьему. В *OpenCV* по умолчанию используется порядок цветов *RGB* или *BGR*?

**Задание 3:** уменьшите вдвое размеры кадра с помощью функции *cv2.resize* перед построением распределения. Обратите внимание на изменение скорости работы программы.

**Задание 4:** добавьте выведение распределений яркостей в двух других каналах. Разберитесь с использованием функции *np.concatenate*, объединяющей несколько массивов в один, и выведите кадры видео и распределения в одно окно.

In [7]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def print_val(val):
    print(val)

def nothing(val):
    pass

#функция построения и рисования распределения
def plot_dist(channel):
    fig, ax = plt.subplots()
    ax.hist(channel.ravel(), 25, [0,256])
    
    fig.canvas.draw()
    dist = np.array(fig.canvas.renderer.buffer_rgba())
    plt.close()
    
    return dist

cv2.namedWindow("frame")

cv2.createTrackbar("value", "frame", 20, 200, nothing)

video_name = "unicycle.mp4"

cam = cv2.VideoCapture(video_name)

skip_frame_reading = False

success, frame_ = cam.read()

while(True):
    if (skip_frame_reading == False):
        success, frame_ = cam.read()
    
    if (success == False):
        print("reading failed")
        
        cam.release()

        cam = cv2.VideoCapture(video_name)
        
        continue
    
    w, h, _ = frame_.shape
    
    frame = cv2.resize(frame_, (h // 2, w // 2))
    
    val = cv2.getTrackbarPos("value", "frame")
    
    frame[:, :, 0] += np.uint8(val)
    
    #########################
    # будет построено распределение яркостей в нулевом канале,
    # то есть frame[:, :, 0]
    
    dist_0 = plot_dist(frame[:, :, 0])
    dist_1 = plot_dist(frame[:, :, 1])
    dist_2 = plot_dist(frame[:, :, 2])

    dists = np.concatenate((dist_0, dist_1, dist_2), axis=1)
        
    wd, hd, _ = dists.shape
    
    dists_resized = cv2.resize(dists, (h // 2, int(w // 2 * h // 2 / hd)))
    
    #обратите внимание, что число каналов в изображении в графиками
    #равно четырем, и его нажно сократить до 3, чтобы
    #сконкатенировать с кадром
    result = np.concatenate((frame, dists_resized[:, :, :3]), axis=0)
    
    cv2.imshow("frame", result)

    #########################

    key = cv2.waitKey(240) & 0xFF
        
    if (key == ord('q')):
        break
    
    if (key == 32):
        skip_frame_reading = not skip_frame_reading

cam.release()
cv2.destroyAllWindows()
cv2.waitKey(10)

reading failed


-1

## Часть 2: цветовые пространства

**Задание 5:** добавьте перевод изображения в цветовое пространство *HSV* до прибавления значения трекбара и обратный перевод после. Как изменяются распределения (которые нужно строить по *RGB* изображению после обратного перевода) при изменении *H*, *S*, *V*?

**Задание 6:** избавьтесь от переполнений, заменив операцию с прибавлением числа на использование функции *cv2.add*. Установите 100 значением по умолчанию для трекбара и добавьте вычитание 100 после считывания, чтобы программа выводила видео без изменений сразу после запуска. Постарайтесь понять, какие изменения происходят с кадрами видео и почему при прибавлении значения к разным каналам. Попробуйте добиться видимости дневного освещения с помощью манипуляций в *HSV*. Попробуйте сделать видео серым.

**Задание 7:** попробуйте уменьшать изображение с помощью *cv2.resize* или задания шага в обращении к массивам (*array[::k]*). Засеките время построения одного распределения и время одного прохода цикла с помощью библиотеки *time*.

In [11]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math

def nothing(val):
    pass

#функция построения и рисования распределения
def plot_dist(channel):
    fig, ax = plt.subplots()
    ax.hist(channel.ravel(), 25, [0,256])
    
    fig.canvas.draw()
    dist = np.array(fig.canvas.renderer.buffer_rgba())
    plt.close()
    
    return dist

cv2.namedWindow("frame")

cv2.createTrackbar("value", "frame", 100, 255, nothing)

video_name = "unicycle.mp4"

cam = cv2.VideoCapture(video_name)

skip_frame_reading = False

success, frame_ = cam.read()

#out = cv2.VideoWriter('board_corners.avi',
#        cv2.VideoWriter_fourcc('M','J','P','G'), 20, (1920, 1080))

while(True):
    if (skip_frame_reading == False):
        success, frame_ = cam.read()
    
    if (success == False):
        print("reading failed")
        
        cam.release()

        cam = cv2.VideoCapture(video_name)
        
        continue
    
    w, h, _ = frame_.shape
    
    frame = cv2.resize(frame_, (h, w))
    
    val = cv2.getTrackbarPos("value", "frame")
    
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    #frame_hsv[:, :, 1] += np.uint8(val - 100)
    
    frame_hsv[:, :, 2] = cv2.add(frame_hsv[:, :, 2], val - 100)
    
    frame = cv2.cvtColor(frame_hsv, cv2.COLOR_HSV2BGR)
        
    dist_0 = plot_dist(frame[::10, ::10, 0])
    dist_1 = plot_dist(frame[::10, ::10, 1])
    dist_2 = plot_dist(frame[::10, ::10, 2])

    dists = np.concatenate((dist_0, dist_1, dist_2), axis=1)
        
    wd, hd, _ = dists.shape
    
    dists_resized = cv2.resize(dists, (h, 2 * int(w // 2 * h // 2 / hd)))
    
    result = np.concatenate((frame, dists_resized[:, :, :3]), axis=0)
        
    #frame[:, :, 1] += (255 - dst) // 3
    
    #cv2.imshow("asca", 255 - dst)
    
    cv2.imshow("frame", result)
    
    #out.write(cv2.cvtColor(255 - dst, cv2.COLOR_GRAY2RGB))
    
    key = cv2.waitKey(240) & 0xFF
        
    if (key == ord('q')):
        break
    
    if (key == 32):
        skip_frame_reading = not skip_frame_reading

out.release()

cam.release()
cv2.destroyAllWindows()
cv2.waitKey(10)

reading failed


-1

## Часть 3: работа с  масками

Можно ли выделить выделить все объекты вместе, при этом не выделяя фон? Возможно это или нет? Попробуйте это сделать.

У разных объектов цвет отличается, и их все нельзя выделить внутри области в пространстве *HSV*, в которой не находился бы фон. Поэтому вместо того, чтобы выделять объекты, выделим фон, а потом инвертируем маску.

**Задание 8:** уменьшите размер изображения, чтобы оно вместе с маской влезало на экран.

**Задание 9:** добавьте печать текущих порогов (со стиранием старых). Выведите маску и исходное изображение в одном окне, настройте цветовой фильтр на фон и измените у трекбаров значения по умолчанию.

**Задание 10:** инвертируйте маску, чтобы объекты отображались на ней в виде белых областей.

**Задание 11:** примените сглаживающий фильтр (*cv2.blur*). Добавьте трекбар для размера ядра фильтра, подойдут значения от *1* (без сглаживания) до достаточно больших, порядка нескольких десятков. Отследите, как изменяется качество выделения объектов при применени сглаживания разной интенсивности

In [14]:
import numpy as np
import cv2
import copy
from IPython.display import clear_output

def nothing(val):
    pass

cv2.namedWindow("frame")

cv2.createTrackbar("lb", "frame",   7, 255, nothing)
cv2.createTrackbar("lg", "frame",  14, 255, nothing)
cv2.createTrackbar("lr", "frame",  78, 255, nothing)
cv2.createTrackbar("hb", "frame",  25, 255, nothing)
cv2.createTrackbar("hg", "frame",  80, 255, nothing)
cv2.createTrackbar("hr", "frame", 185, 255, nothing)
cv2.createTrackbar("ksz", "frame", 0, 20, nothing)

img = cv2.imread("objects.jpg")

while(True):
    frame_ = copy.deepcopy(img)
    
    w, h, _ = frame_.shape
    
    frame = cv2.resize(frame_, (h // 4, w // 4))

    ksz = cv2.getTrackbarPos("ksz", "frame") + 1
    frame = cv2.blur(frame, (ksz, ksz))
    
    lb = cv2.getTrackbarPos("lb", "frame")
    lg = cv2.getTrackbarPos("lg", "frame")
    lr = cv2.getTrackbarPos("lr", "frame")
    hb = cv2.getTrackbarPos("hb", "frame")
    hg = cv2.getTrackbarPos("hg", "frame")
    hr = cv2.getTrackbarPos("hr", "frame")
    
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    mask = cv2.inRange(frame_hsv, (lb, lg, lr), (hb, hg, hr))
    
    mask_3ch = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    res = np.concatenate((frame, 255 - mask_3ch), axis=1)
    
    cv2.imshow("frame", res)
    #cv2.imshow("mask", mask)

    print(lb, lg, lr, hb, hg, hr)
    
    key = cv2.waitKey(240) & 0xFF
    
    clear_output(wait=True)
    
    if (key == ord('q')):
        print(lb, lg, lr, hb, hg, hr)
        break

cv2.destroyAllWindows()
cv2.waitKey(10)

7 14 78 25 80 185


-1

## Часть 4: морфологическая обработка масок

Ознакомьтесь с документацией по морфологическим операциям

https://docs.opencv.org/3.4/d9/d61/tutorial_py_morphological_ops.html

**Задание 12:** попробуйте применить эрозию, наращивание, открытие, закрытие. Обратите внимание на то, что происходит с шумом, с областями внутри маски при больших размерах ядра (около 50).

**Задание 13:** попробуйте скомбинировать несколько морфологических операций, чтобы полностью убрать шум, но при этом маски объектов точно соответствовали объектам.

In [17]:
import numpy as np
import cv2
import copy
from IPython.display import clear_output

def nothing(val):
    pass

cv2.namedWindow("frame")

cv2.createTrackbar("ksz", "frame", 0, 50, nothing)

img = cv2.imread("objects.jpg")

while(True):
    frame_ = copy.deepcopy(img)
    w, h, _ = frame_.shape
    frame = cv2.resize(frame_, (h // 4, w // 4))
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = 255 - cv2.inRange(frame_hsv, (7, 14, 78), (25, 80, 185))
    
    #применение морфологических операций
    ksz = cv2.getTrackbarPos("ksz", "frame") + 1
    kernel = np.ones((ksz, ksz), np.uint8)
    #mask = cv2.erode(mask, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    mask_3ch = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    res = np.concatenate((frame, mask_3ch), axis=1)
    
    cv2.imshow("frame", res)
    
    #out.write(res_resized)
    
    key = cv2.waitKey(240) & 0xFF
    
    clear_output(wait=True)
    
    if (key == ord('q')):
        break

#out.release()

cv2.destroyAllWindows()
cv2.waitKey(10)

-1

## Часть 5: фильтрация связных компонент

Изучите первый ответ по этой ссылке

https://stackoverflow.com/questions/35854197/how-to-use-opencvs-connectedcomponentswithstats-in-python

**Задание 14:** добавьте в код фильтрацию объектов по площади.

**Задание 15:** нарисуйте вокруг объектов, которые остаются после фильтрации, их bounding box-ы (ограничивающие прямоугольники).

**Задание 16:** используя все изученные фильтры и все доступные характеристики областей маски (ширину, высоту и площадь), выделите по отдельности все объекты, то есть только первый, только второй и так далее. Это задание можно выполнять параллельно для разных объектов, например первая группа выделяет левые четыре объекта, а вторая правые четыре.

In [21]:
import numpy as np
import cv2
import copy
from IPython.display import clear_output

def nothing(val):
    pass

cv2.namedWindow("frame")

cv2.createTrackbar("area", "frame", 0, 100, nothing)

img = cv2.imread("objects.jpg")

while(True):
    frame_ = copy.deepcopy(img)
    w, h, _ = frame_.shape
    frame = cv2.resize(frame_, (h // 4, w // 4))
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = 255 - cv2.inRange(frame_hsv, (7, 14, 78), (25, 80, 185))
    
    den_th = cv2.getTrackbarPos("area", "frame") / 100.0
    
    #далее происходит обработка связных компонент
    connectivity = 4
    output = cv2.connectedComponentsWithStats(mask, connectivity, cv2.CV_32S)
    num_labels = output[0]
    labels = output[1]
    stats = output[2]
        
    for i in range(1, num_labels):
        #if (stats[i, cv2.CC_STAT_AREA] < th_area):
        a = stats[i, cv2.CC_STAT_AREA]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        
        density = a / (w * h)

        if (density < den_th):
            mask[np.where(labels == i)] = 0
    
    mask_3ch = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    res = np.concatenate((frame, mask_3ch), axis=1)
    
    cv2.imshow("frame", res)
    
    key = cv2.waitKey(240) & 0xFF
    
    clear_output(wait=True)
    
    if (key == ord('q')):
        break

cv2.destroyAllWindows()
cv2.waitKey(10)

-1

## Часть 6: свободное плавание

**Задание 17:** реализуйте программу, выделяющую лицо человека на кадрах с веб-камеры.

In [ ]:
import cv2

